# Job Shop Problem (车间作业问题)

一个经典的车间调度问题，有多个任务，每个任务由多个作业组成。这些作业需要在多台机器上进行处理。问，如何把这些作业分配到这些机器上，使得完成每个作业的最晚时间尽可能地早。

同时，这些作业还需要满足一些约束。

1. 对于某个任务的多个作业，存在先后关系，必须先处理前面的作业，然后才能处理后面的。
2. 一台机器一次只能处理一个作业。
3. 一个作业一旦开始处理，那么就必须一直加工直到结束。


-----

## 测试数据

一共3台机器，一共3个Job，每个Job的作业如下：

Job0 : [(0, 3), (1, 2), (2, 2)] 

Job1 : [(0, 2), (2, 1), (1, 4)]

Job2 : [(1, 4), (2, 3)]

其中，( m, p ) 记录一个作业的情况，m 表示作业必须在编号 m 的机器上加工， p 表示加工时间是 p 分钟。对于 Job 0， 必须先在机器0上完成持续时间3的作业，才能继续在机器1上完成持续时间2的作业。以此类推。最终需要把所有8个作业都安排到机器上。

一个可行的调度方案如下所示：（并非最优）

![](https://developers.google.cn/static/optimization/images/scheduling/schedule1.png)


-----

## 模型概述

记 $\text{Task}_{i, j}$ 表示第 $i$ 个Job的第 $j$ 个作业。

决策变量：$t_{i,j}$ ，表示 $\text{Task}_{i, j}$ 的开始加工时间。

我们需要满足的约束：

1. 优先级约束：更加靠前的工序一定要更早地加工。以 Job0为例， $t_{0,2} + 2 \leq t_{0,3}$。第2个作业开始加工的时间 + 第2个作业的加工时间 $\leq$ 第3个作业开始加工的时间。
2. 无重叠约束：每个机器不能同时处理两个作业。首先每个机器需要处理的所有作业，我们是知道的，不知道的是处理的先后。那么，我们针对这些作业两两之间进行如下的判断以实现该约束：

> 以 $\text{Task}_{0,2}$ 和 $\text{Task}_{2,1}$ 为例子，它们都在第一个机器上加工。
> 
> 1. 如果 $\text{Task}_{0,2}$ 先加工，那么在他生产的时间内，$\text{Task}_{2,1}$ 一定不加工，所以有：
>
>  $t_{0,2} + 2 \leq t_{2,1}$
>
> 2. 如果 $\text{Task}_{2,1}$ 先加工，那么在他生产的时间内，$\text{Task}_{0,2}$ 一定不加工，所以有：
>
>  $t_{2,1} + 4 \leq t_{0,2}$

上述两者必定满足一个。

基于此，我们用ortools对这个 Job Shop Problem 进行求解。


In [5]:
import collections
from ortools.sat.python import cp_model

In [10]:
def main(jobs_data) -> None:
    """
    Minimal jobshop problem.
    """
    
    # Data.
    # task = (machine_id, processing_time).
    # jobs_data = [  
    #     [(0, 3), (1, 2), (2, 2)],  # Job0
    #     [(0, 2), (2, 1), (1, 4)],  # Job1
    #     [(1, 4), (2, 3)],  # Job2
    # ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job) 
    
    # 假设所有工作只能在一台机器上加工，那么所需要的时间就是所有task的总时间 horizon，
    # 相当于约束每个作业开始加工时间的上界

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple("task_type", "start end interval")
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple(
        "assigned_task_type", "start job index duration"
    )

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix) # 工作开始时间的变量
            end_var = model.new_int_var(0, horizon, "end" + suffix) # 工作结束时间
            interval_var = model.new_interval_var( # 用 interval_var 表示一个工作占用了一个interval
                start_var, duration, end_var, "interval" + suffix
            )
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var
            )
            machine_to_intervals[machine].append(interval_var)

    # for k, v in machine_to_intervals.items():
    #     print(k, v)
    # for k, v in all_tasks.items():
    #     print(k, v)
    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])
        # 这里就是用到add_no_overlap函数的地方，结合interval变量进行使用，一个列表里的intervals不能重叠

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.add(
                all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end
            )

    # Makespan objective.
    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(
        obj_var,
        [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)],
    )
    # add_max_equality 的用法，含义是令前面声明的决策变量obj_var 等于下面列表中最晚的完成时间
    model.minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1],
                    )
                )

        # Create per machine output lines.
        output = ""
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = "Machine " + str(machine) + ": "
            sol_line = "           "

            for assigned_task in assigned_jobs[machine]:
                name = f"job_{assigned_task.job}_task_{assigned_task.index}"
                # add spaces to output to align columns.
                sol_line_tasks += f"{name:15}"

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = f"[{start},{start + duration}]"
                # add spaces to output to align columns.
                sol_line += f"{sol_tmp:15}"

            sol_line += "\n"
            sol_line_tasks += "\n"
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)
    else:
        print("No solution found.")

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")


if __name__ == "__main__":
    jobs_data = [
        [(0, 3), (1, 2), (2, 2), (3, 5), (4,2)],
        [(0, 2), (2, 1), (1, 4), (2, 3), (3,1), (4, 2)],
        [(1, 4), (2, 3), (2, 4), (4, 2), (0, 3)],
        [(1, 2), (3, 4), (2, 3), (0, 2), (1, 1), (4, 3)]
    ]
    
    main(jobs_data)


Solution:
Optimal Schedule Length: 24.0
Machine 0: job_0_task_0   job_1_task_0   job_3_task_3   job_2_task_4   
           [0,3]          [3,5]          [13,15]        [19,22]        
Machine 1: job_2_task_0   job_3_task_0   job_0_task_1   job_1_task_2   job_3_task_4   
           [0,4]          [4,6]          [6,8]          [8,12]         [15,16]        
Machine 2: job_2_task_1   job_1_task_1   job_0_task_2   job_3_task_2   job_2_task_2   job_1_task_3   
           [4,7]          [7,8]          [8,10]         [10,13]        [13,17]        [17,20]        
Machine 3: job_3_task_1   job_0_task_3   job_1_task_4   
           [6,10]         [10,15]        [20,21]        
Machine 4: job_0_task_4   job_2_task_3   job_3_task_5   job_1_task_5   
           [15,17]        [17,19]        [19,22]        [22,24]        


Statistics
  - conflicts: 0
  - branches : 2
  - wall time: 0.005133s
